### Using `perspective.Table`
This notebook illustrates the concepts and usage of `perspective.Table`, Perspective's core component that allows for lightning-fast data loading, update, and transformation.

In [ ]:
from perspective import Table
from datetime import date, datetime
import numpy as np
import pandas as pd

Perspective supports 6 core data types: int, float, str, bool, date, and datetime.

In [ ]:
data = {
    "int": [i for i in range(4)],
    "float": [i * 1.25 for i in range(4)],
    "str": ["a", "b", "c", "d"],
    "bool": [True, False, True, False],
    "date": [date.today() for i in range(4)],
    "datetime": [datetime.now() for i in range(4)]
}

# data can be in row format
rows = [{"a": 1, "b": True}, {"a": 2, "b": False}]

# contain numpy arrays
npdata = {
    "a": np.arange(0, 2),
    "b": np.array(["a", "b"], dtype=object),
    "nullable": np.full(2, np.nan), # perspective handles `None` and `np.nan` values
    "mixed": [None, 1]
}

# or pandas DataFrames
df = pd.DataFrame(npdata)

# or a mixture:
mixed = {
    "a": np.arange(100),
    "b": ["str" for i in range(100)],
    "c": [None for i in range(100)],
    "d": np.full(100, np.nan),
    "e": [datetime.now() for i in range(100)]
}

Perspective schemas can be used as an explicit type mapping for columns - if a schema isn't provided on `__init__`, types will be inferred.

In [ ]:
schema = {
    "int": float,
    "float": int,
    "str": str,
    "bool": bool,
    "date": datetime,
    "datetime": datetime
}

# types can be specified using Python types or string representations
schema2 = {
    "int": "integer",
    "float": "float",
    "str": "string",
    "bool": "boolean",
    "date": "date",
    "datetime": "datetime"
}

Create a `perspective.Table` by passing in a dataset or a schema.

In [ ]:
table = Table(data)

# tables can be created from schema
table2 = Table(schema)
assert table2.size() == 0

# constructing a table with an index, which is a column name to be used as the primary key
indexed = Table(data, index="str")

# or a limit, which is a total cap on the number of rows in the table - updates past `limit` overwite at row 0
limited = Table(data, limit=2)

Tables have their own metadata:

In [ ]:
print("Table has {} rows".format(table.size()))
print("Table columns:", table.columns())
print("Table schema:", table.schema())

`update(data)` can be called on the table instance.

In [ ]:
# you can update all columns
table.update(data)
print("after update:", table.size())

# or however many you'd like
table.update({
    "int": [5, 6, 7],
    "str": ["x", "y", "z"]
})

# but you cannot add new columns through updating - create a new Table instead
try:
    table.update({
        "abcd": [1]
    })
except:
    pass

# updates on unindexed tables always append
print("after append:", table.size())

# updates on indexed tables should include the primary key - the new data overwrites at the row specified by the primary key
indexed.update([{"str": "b", "int": 100}])
print("after indexed partial update:", indexed.size())

# without a primary key, the update appends to the end of the dataset
indexed.update([{"int": 101}])
print("after indexed append:", indexed.size())

`remove([pkeys])` can also be called **on indexed tables** - when provided with an array of primary keys, Perspective removes the rows at those keys.

In [ ]:
indexed.remove(["a", "b"])
print("after remove:", indexed.size())

#### Creating and using Views
A `View` represents an immutable set of transformations on the `perspective.Table`.

##### _Create a view using `table.view()`_

In [ ]:
view = table.view() # a view with zero transformations - returns the dataset as passed in

# view metadata
print("View has {} rows and {} columns".format(view.num_rows(), view.num_columns()))
print(view.schema())

##### _Apply transformations to it_

In [ ]:
pivoted = table.view(row_pivots=["int"], column_pivots=["str"]) # group and split the underlying dataset

aggregated = table.view(row_pivots=["int"], aggregates={"float": "avg"}) # specify aggregations for individual columns

subset = table.view(columns=["float"]) # show only the columns you're interested in

sorted_view = table.view(sort=[["str", "desc"], ["int", "asc"]]) # sort on a specific column, or multiple columns

filtered = table.view(filter=[["int", ">", 2]]) # filter the dataset on a specific value

_Output serialized data from the view_

Perspective outputs data in several formats:
- `to_records`: outputs a list of dictionaries, each of which is a single row
- `to_dict`: outputs a dictionary of lists, each string key the name of a column
- `to_numpy`: outputs a dictionary of numpy arrays
- `to_df`: outputs a `pandas.DataFrame`

In [ ]:
rows = view.to_records()

columnar = view.to_dict()

np_out = view.to_numpy()

df_out = view.to_df()

Data from pivoted or otherwise transformed views reflect the state of the transformed dataset.

In [ ]:
filtered_df = filtered.to_df()
filtered_df

`update()` on the underlying table propagate to views.

In [ ]:
v1 = table.view()
v2 = table.view(row_pivots=["int"])
print("v1 has {} rows and {} columns".format(v1.num_rows(), v1.num_columns()))
print("v2 has {} rows and {} columns".format(v2.num_rows(), v2.num_columns()))

In [ ]:
table.update({"int": [100, 200, 300, 400]})

In [ ]:
print("v1 has {} rows and {} columns".format(v1.num_rows(), v1.num_columns()))
print("v2 has {} rows and {} columns".format(v2.num_rows(), v2.num_columns()))

#### Callbacks

Callback functions can be set on both the `Table` and the `View` instances.

- `View.on_update`: a callback that will be fired on the specified view instance after its underlying table completes processing of updated data. 
    * multiple `on_update` callbacks can be set, and they will be fired in the order in which they were set.
- `View.on_delete`: a callback that will be fired when the view's `delete()` method is called, either manually or by the GC.
    * multiple `on_delete` callbacks can be set.
- `Table.on_delete`: a callback that will be fired when the table's `delete()` method is called, either manually or by the GC.
    * multiple `on_delete` callbacks can be set.

In [ ]:
# define a simple callback
update_counter = 0
def callback():
    global update_counter
    update_counter += 1

new_table = Table(data)
v3 = new_table.view()
v3.on_update(callback)
for i in range(5):
    new_table.update(data)

print("The table was updated {} times".format(update_counter))

In [ ]:
# create an on_delete callback
def deleted():
    print("Finished deleting.")
new_table.on_delete(deleted)
v3.on_delete(deleted)
v3.delete()
new_table.delete()

Callbacks can be removed using the `remove_update` or `remove_delete` methods, both of which take a function to remove.

In [ ]:
counter = 0

def update_callback():
    global counter
    counter += 1
    
def delete_callback():
    print("DELETED")

table4 = Table(data)
view1 = table4.view()

view1.on_update(update_callback)
view1.on_delete(delete_callback)

for i in range(5):
    table4.update(data)
    
# remove the callback
view1.remove_update(update_callback)

for i in range(5):
    table4.update(data) # should no longer trigger callback
    
print("The table was updated {} times".format(counter))

view1.remove_delete(delete_callback)
view1.delete() # should not call the callback